# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire
tol = 1e-12
is_converged = DFTK.ScfConvergenceDensity(tol);

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846733772536                   -0.70    4.8
  2   -7.852306649566       -2.25       -1.54    1.0
  3   -7.852646200815       -3.47       -2.51    3.2
  4   -7.852646676679       -6.32       -3.36    2.0
  5   -7.852646686041       -8.03       -4.68    1.8
  6   -7.852646686725       -9.16       -5.33    3.2
  7   -7.852646686730      -11.36       -6.10    1.2
  8   -7.852646686730      -12.66       -7.77    2.2
  9   -7.852646686730      -14.35       -7.66    2.8
 10   -7.852646686730   +  -15.05       -8.65    2.2
 11   -7.852646686730   +  -14.75       -9.14    1.5
 12   -7.852646686730   +    -Inf      -10.36    2.2
 13   -7.852646686730      -15.05      -10.95    3.0
 14   -7.852646686730   +  -15.05      -11.65    1.8
 15   -7.852646686730   +  -14.75      -12.17    2.0


## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag
---   ---------------   ---------   ---------   ----   ----
  1   -7.846853492606                   -0.70           4.8
  2   -7.852526808357       -2.25       -1.64   0.80    2.0
  3   -7.852635620944       -3.96       -2.74   0.80    1.0
  4   -7.852646460419       -4.96       -3.23   0.80    2.2
  5   -7.852646666935       -6.69       -4.05   0.80    1.0
  6   -7.852646686306       -7.71       -4.70   0.80    1.8
  7   -7.852646686717       -9.39       -5.43   0.80    2.0
  8   -7.852646686729      -10.90       -6.73   0.80    1.8
  9   -7.852646686730      -12.38       -7.60   0.80    2.8
 10   -7.852646686730      -14.57       -8.16   0.80    2.0
 11   -7.852646686730   +    -Inf       -9.17   0.80    1.8
 12   -7.852646686730   +  -15.05       -9.31   0.80    2.2
 13   -7.852646686730   +    -Inf       -9.99   0.80    1.0
 14   -7.852646686730      -15.05      -11.47   0.80    1.2
 15   -7.852646686730      -14.75      -

## Direct minimization

In [4]:
scfres_dm = direct_minimization(basis; tol);

Iter     Function value   Gradient norm 
     0     1.395687e+01     3.864661e+00
 * time: 0.446962833404541
     1     1.071167e+00     1.823008e+00
 * time: 0.6789309978485107
     2    -1.710521e+00     1.901192e+00
 * time: 0.7054078578948975
     3    -3.911697e+00     1.705978e+00
 * time: 0.7415950298309326
     4    -5.393641e+00     1.628686e+00
 * time: 0.7781398296356201
     5    -6.932559e+00     9.102782e-01
 * time: 0.8183658123016357
     6    -7.178674e+00     1.255233e+00
 * time: 0.8466508388519287
     7    -7.642353e+00     9.633604e-01
 * time: 0.8731679916381836
     8    -7.719119e+00     1.122243e+00
 * time: 0.8990168571472168
     9    -7.758808e+00     7.774013e-01
 * time: 0.9253659248352051
    10    -7.781798e+00     8.788101e-01
 * time: 0.9514050483703613
    11    -7.807279e+00     3.977341e-01
 * time: 0.9773948192596436
    12    -7.821200e+00     2.183187e-01
 * time: 1.003161907196045
    13    -7.832327e+00     8.908533e-02
 * time: 1.030306816101

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=1e-1);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846829849715                   -0.70    4.8
  2   -7.852320836523       -2.26       -1.53    1.0


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ, _ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation)
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)
---   ---------------   ---------   ---------
  1   -7.852646686712                   -2.54
  2   -7.852646686730      -10.76       -5.99
  3   -7.852646686730   +    -Inf      -12.68


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 4.837090262825468e-13
|ρ_newton - ρ_scfv| = 8.787592370075528e-13
|ρ_newton - ρ_dm|   = 2.433994865530496e-9
